<a href="https://colab.research.google.com/github/apfmiranda/Self_Consistency/blob/main/Self_Consistency_(Autoconsist%C3%AAncia).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install -q openai

In [4]:
from google.colab import userdata
import openai

# seta a API key
openai.api_key = userdata.get('OPENAI_API_KEY')

# Definição de persona do sistema e histórico de mensagens
messages = [
    {"role": "system", "content": "You are a helpful assistant."},  # seta a persona do modelo, se vai ser educado, formal, sarcástico etc.
]

# SELF-CONSISTENCY
answers = []

LOOPS = 7

prompt = """Retorne no final sempre a string 'Resultado: ' com apenas valor do resultado depois dela, nenhuma operação nem pontuação além do valor.

Pergunta: Se houver 3 carros no estacionamento e chegarem mais 2 carros, quantos carros estarão no estacionamento?

Resposta: Já existem 3 carros no estacionamento. Chegam mais 2. Agora existem 3 + 2 = 5 carros. A resposta é 5.

Resultado: 5

Pergunta: Os patos de Janet botam 16 ovos por dia. Ela come três no café da manhã todas as manhãs e faz muffins para seus amigos todos os dias utilizando mais quatro.
Ela vende o restante por 2 dólares por ovo. Quanto dinheiro ela ganha todos os dias?

Resposta:

Resultado:"""


messages.append({"role": "user", "content": prompt})  # adiciona sua pergunta no histórico do chat

for loop in range(0, LOOPS):
    # Chamada à API
    response = openai.chat.completions.create(
        model = "gpt-3.5-turbo",
        messages = messages,
        max_tokens = 200,
        temperature = 1
    )

    # Extrai a resposta de dentro do JSON de resposta
    answer = response.choices[0].message.content
    print(f"### LOOP", loop, ':\n')
    print(answer)

    start_index = answer.find("Resultado: ") + len("Resultado: ")
    if start_index != -1:
        # Check if the extracted string is empty or contains only whitespace
        extracted_string = answer[start_index:].strip()
        if extracted_string:
            number_answer = extracted_string.split()[0].rstrip('.')
            print("Answer:", number_answer, '\n------')
            answers.append(int(number_answer))  # adiciona a resposta à lista de respostas
        else:
            print("Warning: 'Resultado: ' found, but no value provided.\n------")

print(answers, '\n')

most_frequent_answer = max(set(answers), key=answers.count)
print("Most frequent answer:", most_frequent_answer)

### LOOP 0 :

Janet ganha dinheiro todos os dias através da venda dos ovos restantes. Cada dia ela vende 16 - 3 (para o café da manhã) - 4 (para os muffins) = 9 ovos. Se ela vende cada ovo por 2 dólares, então ela ganha 9 * 2 = 18 dólares por dia.

Resultado: 18
Answer: 18 
------
### LOOP 1 :

Janet ganha dinheiro vendendo ovos que não consome no café da manhã ou usa para fazer muffins para seus amigos. Ela ganha com a venda de 16 - (3 + 4) = 9 ovos por dia. Se ela vende cada ovo por 2 dólares, ela ganha 9 * 2 = 18 dólares por dia.

Resultado: 18
Answer: 18 
------
### LOOP 2 :

Janet ganha dinheiro vendendo os ovos que os patos botam. Ela come 3 ovos no café da manhã e usa mais 4 para fazer muffins, sobrando 16 - 3 - 4 = 9 ovos para vender. A cada dia ela ganha 9 * 2 = 18 dólares com a venda dos ovos.

Resultado: 18
Answer: 18 
------
### LOOP 3 :

Janet ganha dinheiro vendendo 16 - 3 - 4 = 9 ovos por dia.
Então, ela ganha 9 * 2 = 18 dólares por dia.

Resultado: 18
Answer: 18 
------